### Ingest races.csv file

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [2]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [3]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv("/mnt/gualterformula1dl/raw/races.csv")

##### Step 2 - Add ingestion date and race_timestamp to the dataframe

In [4]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, col, lit

In [5]:
races_with_timestamp_df = races_df.withColumn("ingestion_date", current_timestamp()) \
                                  .withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))

##### Step 3 - Select only the columns required & rename as required

In [6]:
races_selected_df = races_with_timestamp_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

##### Write the output to processed container in parquet format

In [7]:
races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet('/mnt/gualterformula1dl/processed/races')